# Infer SOMA - part 2

--------------

## OBJECTIVE:  Infer sub-cellular component #2: SOMA  in order to understand interactome 

To measure shape, position, and size of the cell body -- the soma.    There are a variety of signals from which we could make this inference.  The two most promising are a composite signal including the residual from linear unmixing (`ch = [1, 4, 5,7]`) and a signal derived from the lysosome channel (`ch = 1`).

Dependencies:
The CYTOSOL inference rely on the NUCLEI AND SOMA inference.  Therefore all of the sub-cellular objects rely on this segmentation.





# IMPORTS

In [16]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os
from collections import defaultdict

import numpy as np
import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters, img_as_float
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label
# # package for io 
# from aicsimageio import AICSImage

from napari.utils.notebook_display import nbscreenshot

#import .infer_subc_2d.base
from infer_subc_2d.base import *

viewer = None

--------------

# IMAGE PROCESSING Objective 2:  infer SOMA
  [OUTLINE: Objective #2](#summary-of-objectives)
## summary of steps

INPUT
- channel  1@4.0 ,5@1.0,7@1.0  (1,4,5, and 7:  6/28 pipeline)
- labeled NUCLEI (objective #1)

PRE-PROCESSING
-  scale to max 1.0
- gaussian  Filter window 10  (gauss 10 -> median 5 : 6/28 pipeline)

CORE-PROCESSING
  - watershed from NU, adaptive threshold, minimum cross-entropy
  - threshold smoothing scale: 1 pixel
  - threshold correction factor: .9 (more lenient)
  - lower / upper bounds  (0.0000267,.2)
  - 200 adaptive window
  - log transformed thresholding
  - fill holes
    - NO discard objects on borde
    

- POST-PROCESSING
  - fill holes
  - remove small objects
  - keep only the "most intense" Soma


OUTPUT
- mask of SOMA
- mask of NU (contained by SOMA)


In [2]:

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")
czi_img_folder = data_path/"raw"

list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[5])

test_img_name = img_file_list[5]

img_data, meta_dict = read_input_image(test_img_name)

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [3]:
chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'NU_object'

#NU_object_filen = export_ome_tiff(NU_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

NU_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )

NU_labels = label(NU_object)


# WORKFLOW #1 

Segmentation on a 3 channel composite as per 3/20 pipeline from MCZ


In [4]:
##########################################################################
# DEFAULT PARAMETERS:
#   note that these parameters are supposed to be fixed for the structure
#   and work well accross different datasets
# default_params = defaultdict(str)

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 10
})


################################

# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 4 #2D 
med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter
print(f"median filtering scale is ~ : { [x*y for x,y in zip(scale,med_filter_size_3D)]}")

default_params['z_factor'] = z_factor
default_params['scale'] = scale



median filtering scale is ~ : [0.5804527163320905, 0.3194866073934927, 0.3194866073934927]


In [5]:

###################
# INPUT
###################
struct_img_raw = (4. * img_data[1,:,:,:].copy() + 
                               1. * img_data[5,:,:,:].copy() + 
                               1. * img_data[7,:,:,:].copy() )

# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100


med_filter_size = 15  



In [6]:
###################
# PRE_PROCESSING
###################
#
struct_img = intensity_normalization( struct_img_raw ,  scaling_param=scaling_param)

# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
struct_img = median_filter_slice_by_slice( 
                                                                struct_img,
                                                                size=med_filter_size  )


structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


log_image, d = log_transform( structure_img_smooth ) 
log_image = intensity_normalization(  log_image,  scaling_param=[0] )


edges = filters.scharr(log_image)

composite_soma = intensity_normalization(  edges,  scaling_param=[0] ) + log_image 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [7]:
###################
# CORE_PROCESSING
###################
    
# # no mask
# image_data = np.where(mask, image, np.nan)
# image_data = np.where(mask, image, 0)



#structure_img_smooth = raw_gaussian_filter2D
# this is closer to the original 
bw, _bw_low_level = MO(composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
                                                


In [8]:
# adaptive_window = 200
# if adaptive_window % 2 == 0:
#     adaptive_window += 1
# local_threshold = filters.threshold_sauvola(
#     log_image, window_size=adaptive_window
# )


# # this implimentation doesn't seem to be working despite the fact that I've borrowed from 

# image_data = log_image
# volumetric = True
 
# tolerance=max(np.min(np.diff(np.unique(image_data))) / 2, 0.5 / 65536)
# tolerance=np.min(np.diff(np.unique(image_data))) / 2
# tolerance = None
# #th_method = "Li" #skimage.filters.threshold_li,
# window_size = 200
# th_method=filters.threshold_li
    
# local_threshold = cp_adaptive_threshold( image_data,
#                                                                     th_method, #skimage.filters.threshold_li,
#                                                                     volumetric,
#                                                                     window_size, 
#                                                                     tolerance
#                                                             )

# threshold_correction_factor = 0.9
# threshold_global = filters.threshold_li(image_data)
# corrected_threshold = local_threshold.copy()*threshold_correction_factor

# thresh_min, thresh_max = 0.0000267,.2


# # Constrain the local threshold to be within [0.7, 1.5] * global_threshold. It's for the pretty common case
# # where you have regions of the image with no cells whatsoever that are as large as whatever window you're
# # using. Without a lower bound, you start having crazy threshold s that detect noise blobs. And same for
# # very crowded areas where there is zero background in the window. You want the foreground to be all
# # detected.
# t_min = max(thresh_min, threshold_global * 0.7)
# t_max = min(thresh_max, threshold_global * 1.5)

# corrected_threshold[corrected_threshold < t_min] = t_min
# corrected_threshold[corrected_threshold > t_max] = t_max


# bw_adapt = image_data >= corrected_threshold


In [9]:

###################
# POST_PROCESSING
###################
width = 80  
# discount z direction
#segmented_soma = remove_small_objects(bw, min_size=width*width*1.5, connectivity=1, in_place=False)
#removed_holes = morphology.remove_small_holes(bw, width ** 3 )

#removed_holes = aicssegmentation.core.utils.hole_filling(bw_adapt, hole_min =0. , hole_max=width**2, fill_2d = True) 
removed_holes = aicssegmentation.core.utils.hole_filling(bw, hole_min =0. , hole_max=width**2, fill_2d = True) 

print(f" remove hole size  ~ : { scale[1]*width} microns, scale:{scale}")

width = 45  
cleaned_img = aicssegmentation.core.utils.size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**2, 
                                                         method = "slice_by_slice" ,
                                                         connectivity=1)
print(f" remove small objects  size  ~ : { scale[1]*width} microns, scale:{scale}")

#sobel_image = np.abs(ndi.sobel(struct_img))
# watershed on the sobel limited by the mask
# labels_out = watershed(
#         image=np.abs(ndi.sobel(structure_img_composite)),
#         markers=NU_labels,
#         connectivity=np.ones((3, 3, 3), bool),
#         mask=cleaned_img,
#     )

watershed_mask = np.logical_or(cleaned_img, NU_labels > 0)
inverted_img = 1. - composite_soma

labels_out = watershed(
            connectivity=np.ones((3, 3,3), bool),
            image=inverted_img,
            markers=NU_labels,
            mask=watershed_mask,
            )


# labels_out2 = watershed(
#         image=filters.scharr(composite_soma),
#         markers=NU_labels,
#         connectivity=np.ones((3, 3, 3), bool),
#         mask=cleaned_img,
#     )



 remove hole size  ~ : 6.3897321478698546 microns, scale:(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)
 remove small objects  size  ~ : 3.594224333176793 microns, scale:(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)


In [15]:

#contour_img = [aicssegmentation.core.output_utils.generate_segmentation_contour(labels_out==l) for l in np.unique(labels_out)]
# def generate_segmentation_contour(im):
#     """generate the contour of the segmentation"""

#     bd = np.logical_xor(erosion(im > 0, selem=ball(1)), im > 0)

#     bd = bd.astype(np.uint8)
#     bd[bd > 0] = 255



NameError: name 'viewer' is not defined

In [18]:

# viewer.add_image(
#     struct_img,
#     scale=scale 
# )

if viewer is None:
    viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )

else: 
    viewer.add_image(
        cleaned_img,
        scale=scale
    )


viewer.scale_bar.visible = True

viewer.add_labels(
    labels_out,
    scale=scale 
)


#
#  viewer.add_labels(
#     contour_img,
#     scale=scale 
# )


<Labels layer 'labels_out [1]' at 0x1488dc430>

# DEFINE `infer_SOMA1` function

In [19]:
# copy this to base.py for easy import

def infer_SOMA1(struct_img, NU_labels,  in_params) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_labels: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         

    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    med_filter_size = 15   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    struct_img, d = log_transform( struct_img ) 
    struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )

    struct_img += filters.scharr(struct_img) 
    struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )  


    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.5

    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    out_p["local_adjust"] = local_adjust 
    #                   # # this is not actually applied for this workflow,,,,
    #                    # threshold_correction_factor = 0.9
    #                    # thresh_min, thresh_max = 0.0000267,.2
    #                    
    #                    # threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
    #                    # out_p['threshold_factor'] = threshold_factor
    #                    # out_p['thresh_min'] = thresh_min
    #                    # out_p['thresh_max'] = thresh_max

    ###################
    # POST_PROCESSING
    ###################
    hole_max = 80  
    # discount z direction
    struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_max = 35
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max


    labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_labels,
                mask= np.logical_or(struct_obj, NU_labels > 0),
                )



    retval = (struct_obj,  labels_out, out_p)
    return retval


In [20]:
chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'NU_object'

#NU_object_filen = export_ome_tiff(NU_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

NU_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )
NU_labels = label(NU_object)


###################
# INPUT
###################
struct_img_raw = (4. * img_data[1,:,:,:].copy() + 
                               1. * img_data[5,:,:,:].copy() + 
                               1. * img_data[7,:,:,:].copy() )

# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100
struct_img = intensity_normalization( struct_img_raw ,  scaling_param=scaling_param)


SO_object, SO_label, out_p =  infer_SOMA1(struct_img.copy(), NU_labels, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
['SO_object']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [21]:

chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'SO_object'

SO_object_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

['SO_object']


In [51]:
viewer.add_image(
    SO_object,
    scale=scale
)

viewer.add_labels(
    SO_label,
    scale=scale
)

<Labels layer 'SO_label' at 0x183763ac0>

# WORKFLOW #2

Segmentation on a 4 channel composite as per 6/22 CellProfiler pipeline from MCZ



In [96]:

###################
# INPUT
###################

composite_channels = [1,4,5,7]
#composite_channels = [1,2,3,4] # andy's best guess but maybe need to scale channel 1 (lysosomes) mega
#composite_channels = range(img_data.shape[0]-1)
composite_channels



gaussian_smoothing_sigma = 8
gaussian_smoothing_truncate_range = 3.0

gaussian_smoothing_sigma_3D = [gaussian_smoothing_sigma*scale[2]/x  for x in scale]


print(f"gaussian filtering width (2D) is ~ : { scale[1]*gaussian_smoothing_sigma} microns, scale:{scale}")
print(f"gaussian filtering width (3d) is ~ : {gaussian_smoothing_sigma_3D} pixels; :{[x*y for x,y in zip(scale,gaussian_smoothing_sigma_3D)]} microns")

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(img_data[1,:,:,:]) #  [0.0, 8]
#truncate_intensity = raw_soma.mean()+raw_soma.std()*3
raw_soma_linear = intensity_normalization(  img_data[composite_channels,:,:,:].copy(), scaling_param=[0] ).sum(axis=0)
raw_soma_linear = intensity_normalization(  raw_soma_linear, scaling_param=[0] )


###################
# PRE_PROCESSING
###################

struct_img = raw_soma_linear.copy()
# non-linear scaling for the aggregate tested below
# # scale_parameters = [[0.0, 24.5],
# #                                     [0.0, 35.0],
# #                                     [0.5, 15.0],
# #                                     [1.5, 10.0]]
# # hold_it = []

# smooth1 = aicssegmentation.core.pre_processing_utils.edge_preserving_smoothing_3d(
#                                     struct_img,
#                                     numberOfIterations = 10,
#                                     conductance = 1.2,
#                                     timeStep = 0.0625,
#                                     spacing= list(scale) )


# smoothing with gaussian filter
# 3D alternative: image_smoothing_gaussian_3d(...)
gaussian_smoothing_sigma = 3
gaussian_smoothing_truncate_range = 3.0

med_filter_size = 9
#structure_img_median = ndi.median_filter(struct_img,    size=med_filter_size  )

structure_img_median = median_filter_slice_by_slice( 
                                                                struct_img,
                                                                size=med_filter_size )

# log_image, d = log_transform( sc_median ) 
# log_image = intensity_normalization(  log_image,  scaling_param=[0] )


structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   structure_img_median,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )

edges = filters.scharr(struct_img)



gaussian filtering width (2D) is ~ : 0.6389732147869854 microns, scale:(0.5804527163320905, 0.07987165184837318, 0.07987165184837318)
gaussian filtering width (3d) is ~ : [1.1008187175429014, 8.0, 8.0] pixels; :[0.6389732147869854, 0.6389732147869854, 0.6389732147869854] microns
mean intensity of the stack: 641.5999045901829
the standard deviation of intensity of the stack: 2144.942904845627
0.9999 percentile of the stack intensity is: 49648.38039997965
minimum intensity of the stack: 0
maximum intensity of the stack: 65535
suggested upper range is 23.0, which is 49975.2867160396
suggested lower range is 0.0, which is 641.5999045901829
So, suggested parameter for normalization is [0.0, 23.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value
intensity normalization: min-max n

In [46]:
list(scale)

viewer.add_image(
    structure_img_smooth,
    scale=scale
)

viewer.add_labels(
    NU_labels,
    scale=scale
)

<Labels layer 'NU_labels' at 0x17c88cfd0>

In [97]:

# ##########################################################################
# PARAMETERS:

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img) #  [0.5, 12.5]
aicssegmentation.core.pre_processing_utils.suggest_normalization_param(structure_img_smooth) # [0.5, 9.0]
aicssegmentation.core.pre_processing_utils.suggest_normalization_param(structure_img_median) # [0.5, 9.0]
# [0.0, 23.5]

#intensity_norm_param = [0.5, 15]
intensity_norm_param = [0]
gaussian_smoothing_sigma = 5
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100
#structure_img_smooth = raw_gaussian_filter2D
# this is closer to the original 
bw, _bw_low_level = MO(structure_img_smooth, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.25, 
                                                return_object=True,
                                                dilate=True)
                                                




###################
# POST_PROCESSING
###################
width = 5  
# discount z direction
#segmented_soma = remove_small_objects(bw, min_size=width*width*1.5, connectivity=1, in_place=False)
removed_holes = morphology.remove_small_holes(bw, width ** 3 )

width = 6  
cleaned_img = aicssegmentation.core.utils.size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**3, 
                                                         method = "3D", #"slice_by_slice" 
                                                         connectivity=1)

#sobel_image = np.abs(ndi.sobel(struct_img))
# watershed on the sobel limited by the mask


labels_out = watershed(
        image=np.abs(ndi.sobel(structure_img_smooth)),
        markers=NU_labels,
        connectivity=np.ones((3, 3, 3), bool),
        mask=cleaned_img,
    )



mean intensity of the stack: 0.022594163479725156
the standard deviation of intensity of the stack: 0.035176025031434015
0.9999 percentile of the stack intensity is: 0.562098829807297
minimum intensity of the stack: 6.00946325238002e-09
maximum intensity of the stack: 1.0
suggested upper range is 15.5, which is 0.5678225514669525
suggested lower range is 0.5, which is 0.0050061509640081485
So, suggested parameter for normalization is [0.5, 15.5]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value
mean intensity of the stack: 0.02121846708738522
the standard deviation of intensity of the stack: 0.03109671695313
0.9999 percentile of the stack intensity is: 0.49197553144795597
minimum intensity of the stack: 0.005571961371672658
maximum intensity of the stack: 0.7053786193844717


In [99]:

labels_out = watershed(
        image=np.abs(ndi.sobel(structure_img_smooth)),
        markers=NU_labels,
        connectivity=np.ones((3, 3, 3), bool),
        mask=cleaned_img,
    )


viewer.add_image(
    structure_img_smooth,
    scale=scale 
)

viewer.add_image(
    cleaned_img,
    scale=scale
)
viewer.scale_bar.visible = True

viewer.add_labels(
    labels_out,
    scale=scale 
)



<Labels layer 'labels_out [1]' at 0x193b5beb0>

# DEFINE `infer_SOMA2` function

In [100]:
# copy this to base.py for easy import

def infer_SOMA2(struct_img, NU_labels,  in_params) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_labels: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    med_filter_size = 9   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 3.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    #    edges = filters.scharr(struct_img)
    # struct_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )
    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.25

    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    out_p["local_adjust"] = local_adjust 

    ###################
    # POST_PROCESSING
    ###################
    hole_max = 80  
    # discount z direction
    struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_max = 35
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    labels_out = watershed(
                                                image=np.abs(ndi.sobel(structure_img_smooth)),
                                                markers=NU_labels,
                                                connectivity=np.ones((3, 3, 3), bool),
                                                mask= np.logical_or(struct_obj, NU_labels > 0),
                                                )


    retval = (struct_obj,  labels_out, out_p)
    return retval


In [101]:
chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'NU_object'

#NU_object_filen = export_ome_tiff(NU_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

NU_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )
NU_labels = label(NU_object)

###################
# INPUT
###################

composite_channels = [1,4,5,7]
raw_soma_linear = intensity_normalization(  img_data[composite_channels,:,:,:].copy(), scaling_param=[0] ).sum(axis=0)
#struct_img = intensity_normalization(  raw_soma_linear, scaling_param=[0] )


SO_object, SO_label, out_p =  infer_SOMA2(raw_soma_linear.copy(), NU_labels, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper
# possibly need to do some post-post-processing to make suer that there is only a single SO_Object?




chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'SO_object2'

SO_object_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

# test: does this export work?
object_name = 'SO_label2'
SO_label_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
['SO_object']
['SO_label']


The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


# WORKFLOW #3

Segmentation on the log-scaled Lysosome signal and aggressively filling holes.



In [64]:

###################
# INPUT
###################

composite_channels = [1]

struct_img_raw = intensity_normalization(  img_data[1,:,:,:].copy(), scaling_param=[0] )



#####################
# PRE=PROCESSING
#####################
# struct_img_smooth = aicssegmentation.core.pre_processing_utils.edge_preserving_smoothing_3d(
#                                     struct_img_raw,
#                                     numberOfIterations = 10,
#                                     conductance = 1.2,
#                                     timeStep = 0.0625,
#                                     spacing= list(scale) )
# # resulted in some negative pixels
#struct_img_smooth = intensity_normalization(  struct_img_smooth, scaling_param=[0] )
struct_img_smooth = struct_img_raw
med_filter_size = 3   
# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
struct_img_smooth2 = median_filter_slice_by_slice( 
                                                                struct_img_smooth,
                                                                size=med_filter_size  )
out_p["median_filter_size"] = med_filter_size 

gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
struct_img_smooth3 = image_smoothing_gaussian_slice_by_slice(   struct_img_smooth2,
                                                                                                    sigma=gaussian_smoothing_sigma,
                                                                                                    truncate_range = gaussian_smoothing_truncate_range
                                                                                                )
out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range


log_image, d = log_transform( struct_img_smooth2 ) 

composite_img = intensity_normalization( log_image + filters.scharr(log_image),  scaling_param=[0] )  


intensity normalization: min-max normalization with NO absoluteintensity upper bound


GradientAnisotropicDiffusionImageFilter (0x7faf991960e0): Anisotropic diffusion unstable time step: 0.0625
Stable time step for this image must be smaller than 0.00499198

GradientAnisotropicDiffusionImageFilter (0x7faf991960e0): Anisotropic diffusion unstable time step: 0.0625
Stable time step for this image must be smaller than 0.00499198

GradientAnisotropicDiffusionImageFilter (0x7faf991960e0): Anisotropic diffusion unstable time step: 0.0625
Stable time step for this image must be smaller than 0.00499198

GradientAnisotropicDiffusionImageFilter (0x7faf991960e0): Anisotropic diffusion unstable time step: 0.0625
Stable time step for this image must be smaller than 0.00499198

GradientAnisotropicDiffusionImageFilter (0x7faf991960e0): Anisotropic diffusion unstable time step: 0.0625
Stable time step for this image must be smaller than 0.00499198

GradientAnisotropicDiffusionImageFilter (0x7faf991960e0): Anisotropic diffusion unstable time step: 0.0625
Stable time step for this image m

NameError: name 'istruct_img_smooth' is not defined

In [91]:



###################
# CORE_PROCESSING
###################
local_adjust = 0.5

struct_obj, _bw_low_level = MO(log_image, 
                                            global_thresh_method='ave', 
                                            object_minArea=low_level_min_size, 
                                            extra_criteria=True,
                                            local_adjust= local_adjust, 
                                            return_object=True,
                                            dilate=True)

out_p["local_adjust"] = local_adjust 

# # this is not actually applied for this workflow,,,,
# threshold_correction_factor = 0.9
# thresh_min, thresh_max = 0.0000267,.2

# threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
# out_p['threshold_factor'] = threshold_factor
# out_p['thresh_min'] = thresh_min
# out_p['thresh_max'] = thresh_max


###################
# POST_PROCESSING
###################
hole_max = 100  
# discount z direction
struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
out_p['hole_max'] = hole_max

small_object_max = 55
struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                        min_size= width**3, 
                                                        method = "slice_by_slice" ,
                                                        connectivity=1)
out_p['small_object_max'] = small_object_max


# labels_out = watershed(
#             connectivity=np.ones((3, 3,3), bool),
#             image=1. - log_image,
#             markers=NU_labels,
#             mask= np.logical_or(struct_obj, NU_labels > 0),
#             )
# labels_out1 = watershed(
#             connectivity=np.ones((3, 3,3), bool),
#             image=np.abs(filters.scharr(log_image)),
#             markers=NU_labels,
#             mask= np.logical_or(struct_obj, NU_labels > 0),
#             )
labels_out1b = watershed(
            connectivity=np.ones((3, 3,3), bool),
            image=np.abs(filters.sobel(log_image)),
            markers=NU_labels,
            mask= np.logical_or(struct_obj, NU_labels > 0),
            )

# labels_out2 = watershed(
#             connectivity=np.ones((3, 3,3), bool),
#             image=composite_img,
#             markers=NU_labels,
#             mask= np.logical_or(struct_obj, NU_labels > 0),
#             )



# labels_out3 = watershed(
#             connectivity=np.ones((3, 3,3), bool),
#             image=log_image,
#             markers=NU_labels,
#             mask= np.logical_or(struct_obj, NU_labels > 0),
#             )




intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [92]:


viewer.add_image(
    composite_img,
    scale=scale
)

viewer.add_image(
    struct_obj,
    scale=scale
)

viewer.add_image(
    log_image,
    scale=scale
)

# viewer.add_labels(
#     labels_out,
#     scale=scale
# )
# viewer.add_labels(
#     labels_out1,
#     scale=scale
# )
# viewer.add_labels(
#     labels_out2,
#     scale=scale
# )
viewer.add_labels(
    labels_out1b,
    scale=scale
)
# viewer.add_labels(
#     labels_out3,
#     scale=scale
# )

<Labels layer 'labels_out1b [1]' at 0x18ef4b4f0>

# DEFINE `infer_SOMA3` function

In [ ]:

def infer_SOMA3(struct_img, NU_labels,  in_params) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_labels: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    med_filter_size = 3   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    struct_img, d = log_transform( struct_img ) 
    struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )

    struct_img += filters.scharr(struct_img) 
    struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )  


    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.5

    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    out_p["local_adjust"] = local_adjust 
    ###################
    # POST_PROCESSING
    ###################
    hole_max = 100  
    # discount z direction
    struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_max = 55
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    labels_out = watershed(
                                                image=np.abs(ndi.sobel(structure_img_smooth)),
                                                markers=NU_labels,
                                                connectivity=np.ones((3, 3, 3), bool),
                                                mask= np.logical_or(struct_obj, NU_labels > 0),
                                                )


    retval = (struct_obj,  labels_out, out_p)
    return retval


In [ ]:
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper

chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'NU_object'

#NU_object_filen = export_ome_tiff(NU_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

NU_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )
NU_labels = label(NU_object)

###################
# INPUT
###################

composite_channels = [1,4,5,7]
raw_soma_linear = intensity_normalization(  img_data[composite_channels,:,:,:].copy(), scaling_param=[0] ).sum(axis=0)
#struct_img = intensity_normalization(  raw_soma_linear, scaling_param=[0] )


SO_object, SO_label, out_p =  infer_SOMA3(raw_soma_linear.copy(), NU_labels, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper
# possibly need to do some post-post-processing to make suer that there is only a single SO_Object?




chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'SO_object3'

SO_object_filen = export_ome_tiff(SO_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

['SO_object']


The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


## POST POST-PROCESSING

Find the label with the brightest florescence..

Keep the Nuclei with that label.  
Mask all the other nuclei.

re-label the soma with watershed using the single.


also try:
    find the center of mass of each nuclei.  Use those as seeds for the soma...



In [ ]:

# keep the "SOMA" which contains the highest total signal
all_labels = np.unique(labels_out)[1:]

total_flourescence = [ struct_img[labels_out == label].sum() for label in all_labels]
# combine NU and "labels" to make a SOMA
keep_label = all_labels[np.argmax(total_flourescence)]
keep_label, total_flourescence

if viewer is None:
    viewer = napari.view_image(
        composite_soma,
        scale=scale,
        blending='additive',
        colormap='magenta',
    )

else:
    viewer.add_image(
        composite_soma,
        scale=scale 
    )

viewer.add_image(
    log_image,
    scale=scale 
)
